In [ ]:
import pandas as pd
import numpy as np


questions = pd.read_csv('hw2/questions.csv')
lectures = pd.read_csv('hw2/lectures.csv')
train = pd.read_csv('hw2/train.csv')

In [ ]:
print(train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101230332 entries, 0 to 101230331
Data columns (total 10 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   row_id                          int64  
 1   timestamp                       int64  
 2   user_id                         int64  
 3   content_id                      int64  
 4   content_type_id                 int64  
 5   task_container_id               int64  
 6   user_answer                     int64  
 7   answered_correctly              int64  
 8   prior_question_elapsed_time     float64
 9   prior_question_had_explanation  object 
dtypes: float64(1), int64(8), object(1)
memory usage: 7.5+ GB

Столбец prior_question_had_explanation, судя по названию, должен иметь тип boolean


In [ ]:
train['prior_question_had_explanation'] = train['prior_question_had_explanation'].astype('boolean')
print(train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101230332 entries, 0 to 101230331
Data columns (total 10 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   row_id                          int64  
 1   timestamp                       int64  
 2   user_id                         int64  
 3   content_id                      int64  
 4   content_type_id                 int64  
 5   task_container_id               int64  
 6   user_answer                     int64  
 7   answered_correctly              int64  
 8   prior_question_elapsed_time     float64
 9   prior_question_had_explanation  boolean
dtypes: boolean(1), float64(1), int64(8)
memory usage: 7.0 GB

Тем самым уменьшился объем используемой памяти для датасета

In [ ]:
print(f'{train.user_id.nunique()} уникальных пользователей')

393656 уникальных пользователей

In [ ]:
print(f'Кол-во уникальных  id {train.content_id.nunique()} в датасете, из которых {train[train.content_type_id == False].content_id.nunique()} - вопросы.')

Кол-во уникальных  id 13782 в датасете, из которых 13523 - вопросы.

In [ ]:
print(f'Имеется {train.task_container_id.nunique()} уникальных пакетов вопросов или лекций.')

Имеется 10000 уникальных пакетов вопросов или лекций.

In [ ]:
print(f'Из {train.user_id.nunique()} юзеров имеется {train[train.timestamp == 0].user_id.nunique()} юзеров с timestamp = 0.')

Из 393656 юзеров имеется 393656 юзеров с timestamp = 0.

In [ ]:
correct = train[train.answered_correctly != -1].answered_correctly.value_counts(ascending=True)
user_percent = train[train.answered_correctly != -1].groupby('user_id')['answered_correctly'].agg(Mean='mean', Answers='count')
print(f'Наибольшее кол-во отвеченных вопросов пользователем: {user_percent.Answers.max()}')

Наибольшее кол-во отвеченных вопросов пользователем: 17609

# ВОПРОСЫ

In [ ]:
print(questions.head())

question_id  bundle_id  correct_answer  part            tags
0            0          0               0     1   51 131 162 38
1            1          1               1     1       131 36 81
2            2          2               0     1  131 101 162 92
3            3          3               0     1  131 149 162 29
4            4          4               3     1    131 5 162 38
(13523, 5)

In [ ]:
tags = [x.split() for x in questions[questions.tags != "nan"].tags.values]
tags = [item for elem in tags for item in elem]
tags = set(tags)
tags = list(tags)
print(f'Кол-во различных тэгов: {len(tags)}')

Кол-во различных тэгов: 188

Кол-во верно и неверно данных ответов на вопросы с градацией по тэгам

In [ ]:
tags_list = [x.split() for x in questions.tags.values]
questions['tags'] = tags_list


correct = train[train.answered_correctly != -1].groupby(["content_id", 'answered_correctly'], as_index=False).size()
correct = correct.pivot(index= "content_id", columns='answered_correctly', values='size')
correct.columns = ['Wrong', 'Right']
correct = correct.fillna(0)
correct[['Wrong', 'Right']] = correct[['Wrong', 'Right']].astype(int)
questions = questions.merge(correct, left_on = "question_id", right_on = "content_id", how = "left")


tags_df = pd.DataFrame()
for x in range(len(tags)):
    df = questions[questions.tags.apply(lambda l: tags[x] in l)]
    df1 = df.agg({'Wrong': ['sum'], 'Right': ['sum']})
    df1['Total_questions'] = df1.Wrong + df1.Right
    df1['Question_ids_with_tag'] = len(df)
    df1['tag'] = tags[x]
    df1 = df1.set_index('tag')
    tags_df = tags_df.append(df1)

tags_df[['Wrong', 'Right', 'Total_questions']] = tags_df[['Wrong', 'Right', 'Total_questions']].astype(int)
tags_df['Percent_correct'] = tags_df.Right/tags_df.Total_questions
tags_df = tags_df.sort_values(by = "Percent_correct")

print(tags_df.head())

Wrong   Right  Total_questions  Question_ids_with_tag  Percent_correct
tag                                                                         
24   157631   98982           256613                     17         0.385725
23   261611  204293           465904                     11         0.438487
19   173602  136367           309969                     57         0.439938
151  272267  264913           537180                     16         0.493155
167  170386  170681           341067                     11         0.500432

# ЛЕКЦИИ

In [ ]:
print(lectures.head())

lecture_id  tag  part           type_of
0          89  159     5           concept
1         100   70     1           concept
2         185   45     6           concept
3         192   79     5  solving question
4         317  156     5  solving question

In [ ]:
print(f'Кол-во лекций: {lectures.shape[0]}')

Кол-во лекций: 418

In [ ]:
print(lectures.type_of.value_counts())

concept             222
solving question    186
intention             7
starter               3

In [ ]:
batch_lect = train.groupby(["task_container_id", "answered_correctly"]).size().unstack()
batch_lect.columns = ['Lecture', 'Wrong', 'Right']
batch_lect['Lecture'] = batch_lect['Lecture'].fillna(0)
batch_lect = batch_lect.astype('Int64')
batch_lect['Percent_correct'] = batch_lect.Right/(batch_lect.Wrong + batch_lect.Right)
batch_lect['Percent_lecture'] = batch_lect.Lecture/(batch_lect.Lecture + batch_lect.Wrong + batch_lect.Right)
batch_lect = batch_lect.sort_values(by = "Percent_lecture", ascending = False)

print(f'Наибольшее кол-во лекций, просмотренных в рамках одного task_container_id: {batch_lect.Lecture.max()}.')
print(batch_lect.head())

Наибольшее кол-во лекций, просмотренных в рамках одного task_container_id: 5143.
                   Lecture  Wrong  Right  Percent_correct  Percent_lecture
task_container_id                                                         
9096                     8     57    191         0.770161          0.03125
270                   2265  26105  52352          0.66727         0.028059
477                   1428  16166  33427         0.674027         0.027988
253                   2343  27380  54798         0.666821         0.027721
351                   1835  21225  43158         0.670332         0.027711